In [59]:
def  getPdfLink(title  ): 
    print("step 1,login into ucalgary...\n")
    time.sleep(5)
    driver.get("https://cas.ucalgary.ca/cas/login?service=https://portal.my.ucalgary.ca/psp/paprd/?cmd=start&ca.ucalgary.authent.ucid=true")
    driver.find_element(By.NAME,'username').clear()
    driver.find_element(By.NAME,'password').clear()
    driver.find_element(By.NAME,'username').send_keys(USERNAME)
    driver.find_element(By.NAME,'password').send_keys(PASSWORD)
    time.sleep(5)
    driver.find_element(By.ID,'signinbutton').click() 
    url = driver.current_url  
    print("step 1,login finished url:\n\n",url)
    
    
    print("title...",title)
    ## input article name to search box and then search it 
    print("step 2,use article name to search published file in ucalgary library...\n ")
    time.sleep(5)
    driver.get('https://library.ucalgary.ca/') 
    time.sleep(2) 
    driver.find_element(By.XPATH, '//*[@id="search"]').clear()
    driver.find_element(By.XPATH, '//*[@id="search"]').send_keys(title)  
    #driver.maximize_window()
    #original_window = driver.current_window_handle 
    time.sleep(5)  
    driver.find_element("xpath",  '//*[@id="unified"]').submit()    
    time.sleep(5)
    driver.switch_to.window(driver.window_handles[1]) 
    print(driver.window_handles)
    url = driver.current_url  
    print("step 2,after search  url:\n",url,"\n")

    ## obtain the pdf download  link   
    pubauthors = ""
    time.sleep(5) 
    try:
        sta = driver.find_element(By.XPATH, '//*[@class="browzine-direct-to-pdf-link"]') 
        publink = sta.get_attribute("href") 
        print(publink)
    except:
        publink=""
        pass
    
    print("authors lists: ....\n" )
    try:
        time.sleep(3)
        sta = driver.find_element(By.XPATH, '//*[@id="SEARCH_RESULT_RECORDID_cdi_proquest_miscellaneous_2605226373"]/div[3]/prm-brief-result/div[1]/span[1]/span[2]/prm-highlight/span') 
        time.sleep(3)
        pubauthors = sta.text
        print("authors lists:--- \n",pubauthors)
    except:
        pubauthors=""
        pass
    print("step 2,finished, artile url in html page:\n",publink,"\n")  


    ## because the former link is not the final pdf link, so need to jump to and get the final download link
    if (publink == ""):
        return url,pubauthors
        
    print("step 3,find the real pdf file url...\n")
    time.sleep(2)
    driver.get(publink) 
    url = driver.current_url 
    print("pdf url:\n",url) 
    time.sleep(5)
    url = driver.current_url 
    print("step 3,pdf final url:===\n",url)   
    time.sleep(5) 
    
    ##  download file 
    print("step 4,save file ...\n") 
    filename = PDF_DIR+"p"+"5.pdf"
    print(filename,"\n")
    wget.download(url,filename)  
    time.sleep(10)
    
    print("\nstep 5,clear ...\n\n") 
    driver.close()
    #driver.switch_to.window(driver.window_handles[0])  
    
    return url,pubauthors

 

In [60]:
import pandas as pd  
from selenium.webdriver import Chrome
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.firefox.options import Options
import time
import wget
import ssl 

## Parameters for ucalgary login
USERNAME = 'huaien.gu'
PASSWORD = 'Sask2ucalg$'
PDF_DIR  = "E:/Ucalgary/internship-proj/rcode/pdf files/"

 
## get links for published article files
rdata   = pd.read_csv("biorxiv-pdf.csv",encoding="latin-1")  

#use selenium for pseudo element extraction, but slow  
numlink = len(rdata["link"])  
url     = [0 for i in range(numlink)]  
publink = [0 for i in range(numlink)]  
pubpdflink = [0 for i in range(numlink)]  
pubauthors = [0 for i in range(numlink)]  
 

## using firefox  
options = webdriver.FirefoxOptions()
options.add_argument('-headless')
driver = webdriver.Firefox(options=options) 
  
print( "getting published paper links... \n")
ssl._create_default_https_context = ssl._create_unverified_context

# get published pdf link from website, but it is not the true website for files. after input, the url could jump to real url 
for i in range(0,numlink) :
     url[i] = rdata["link"][i]    
     driver.get(url[i])  
     publink[i]=driver.find_element("xpath",  '//div[@class="pub_jnl"]/a').get_attribute('href')
     if(i%10 == 0):
            print("total is ",numlink, " finished ",i)
        
publink = [c.replace('https://www.biorxiv.org/about/FAQ#unrefereed','') for c in publink] 
print( "getting published paper links...finished \n")

save to file biorxiv.csv, adding published paper link 
rdata["pub_link"] = publink   
rdata["published"] = 1
rdata.loc[( pd.isna(rdata["pub_link"]) ), "published"] = "0"
rdata["published"]
rdata.to_csv("biorxiv-pdf2.csv" )


## we get all published article linke above
## but we can not use above links to download published files directly, because 
## 1) we need to pay for some websites 
## 2) different website has different format, it is difficult to code

## the following code is another methods, to find the true url link from ucalgary librrary,but not for all articles
## the methods using firefox, python selenium 
## 1) use my username and password login into ucalgary
## 2) use article's name to find in library 
## 3）find article's url in html and true url in library or other place
## 4) save the true links for later download 


## login ucalgary  
rdata   = pd.read_csv("biorxiv-pdf2.csv",encoding="latin-1")  
 
    
#for i in range(0, len(rdata)):  
for i in range(0, 5):  
    if(pd.isna(rdata["pub_link"][i]) ):
        pubpdflink[i] = ""
        pubauthors[i] = ""
        continue
    else:
        gtitle = rdata["title"][i] 
        rv = getPdfLink(gtitle )
        pubpdflink[i] = rv[0]
        pubauthors[i] = rv[1]
        print("pubpdflink[i],pubauthors[i] ...",pubpdflink[i],pubauthors[i] )
        if(i%5 == 0):
            print("total ",numlink," processed ... ",i,"\n")  


# save to file biorxiv.csv, adding pdf link in library  
# rdata["pubauthors"] = pubauthors 
# rdata.to_csv("biorxiv-pdf2.csv" )


getting published paper links... 

step 1,login into ucalgary...

step 1,login finished url:

 https://portal.my.ucalgary.ca/psp/paprd/EMPLOYEE/EMPL/h/?tab=IS_SSS_TAB
title... WgLink: reconstructing whole-genome viral haplotypes using L0 + L1-regularization
step 2,use article name to search published file in ucalgary library...
 
['e5417c1b-f2a7-4b6e-987f-510bc0ef0e46', '0928b054-b4ee-4623-8706-03e52c00ef81']
step 2,after search  url:
 https://ucalgary.primo.exlibrisgroup.com/discovery/search?vid=01UCALG_INST:UCALGARY&tab=QuickSearch&query=any,contains,WgLink:%20reconstructing%20whole-genome%20viral%20haplotypes%20using%20L0%20%2B%20L1-regularization&search_scope=EVERYTHING 

https://libkey.io/libraries/458/articles/436065964/full-text-file?utm_source=api_602
authors lists: ....

step 2,finished, artile url in html page:
 https://libkey.io/libraries/458/articles/436065964/full-text-file?utm_source=api_602 

step 3,find the real pdf file url...

pdf url:
 https://libkey.io/libraries/458

NoSuchElementException: Message: Unable to locate element: [name="username"]
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:188:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:400:5
element.find/</<@chrome://remote/content/marionette/element.js:292:16
